In [1]:
library("xgboost")
library("Matrix")
library('Ckmeans.1d.dp')
library('lightgbm')



Attaching package: ‘lightgbm’


The following object is masked from ‘package:xgboost’:

    slice




## Read data and process data labels

In [2]:
time_matrix <- matrix(0,ncol = 3, nrow =4)
colnames(time_matrix) <- c("user_time", "system_time", "elapsed_time")
start_time = Sys.time()

In [3]:
data=read.csv('Quarterly_features_matrix.csv')
data=data[,2:(dim(data)[2]-6)]

In [4]:
dim(data)

[1] 24000    21

In [5]:
head(data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,⋯,trough,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,4,1,4,0.9492015,1.440272e-05,-3.801739,-1.3040882,-0.4829440,0.4354882,0.3478599,⋯,1,0.5299121,0.8423074,2.490487,-0.51403516,0.9484577,-0.7896943,1.7947057,0.5618988,25
2,4,1,4,0.9332456,2.024615e-05,-3.528016,-1.5740960,-0.3277471,0.3143940,0.1373249,⋯,1,0.3641703,0.8571066,2.440089,-0.29175138,0.2427210,-0.6669261,0.7095412,0.5378848,25
3,4,1,4,0.9753689,3.173153e-06,-4.202105,-1.3102058,-0.3300377,0.2486809,0.1689488,⋯,1,0.2498508,0.8961490,2.733696,-0.24714224,0.2873822,-0.7152289,0.8909252,0.6050959,25
4,4,1,4,0.9597504,7.178017e-06,-4.163362,-0.7087874,-0.2787477,0.4831910,0.1164472,⋯,1,0.4809287,0.8920654,2.726705,-0.20462399,0.2600985,-0.6048727,0.6906562,0.6227230,25
5,4,1,4,0.9770473,1.682990e-07,-5.666607,0.6009075,-0.5379779,0.5748836,0.9582223,⋯,1,0.5409435,0.5996603,2.068462,-0.14366107,2.4828596,-0.2911521,1.9555069,0.7981635,59
6,4,1,4,0.9884024,1.093815e-07,4.960507,-1.0952883,-0.1131941,0.3831215,0.6847558,⋯,1,0.5608638,0.8887061,2.856883,-0.07494233,0.7650798,-0.5923500,2.3289573,0.6350911,59


In [6]:
dlist= load('Quarterly_ets_datalist.RData')
datalist=eval(parse(text = dlist ))
res=datalist[[1]]
MASE=res[,,,6]
m=5

In [7]:
dim(MASE)

[1] 24000     5     4

In [8]:
whichmin<-function(x){
    minx=min(x[x>0])
    loc=which(x==minx)[1]-1
    loc
}

meanunique=function(x)
    {
    mean(unique(x))
}

In [9]:
nanum=c()
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    a=apply(count,1,min)
    if(sum(is.na(a))>0)
        {
        nanum=append(nanum,i)
    }
    }

In [10]:
nanum

[1]   158   159   160   208   237   244   255   259   266   301   303   314
  [13]   322   327   330   340  1081  1201  1558  1564  1576  1579  1582  1585
  [25]  1586  1615  1626  1627  1629  1630  1639  1641  1644  1655  1656  1659
  [37]  1704  1709  1737  1744  1745  1751  1754  1756  1759  1763  1767  1769
  [49]  1773  1776  1778  1779  1784  1787  1788  1789  1792  1793  1794  1802
  [61]  1828  1855  1867  1881  1890  1898  1899  1902  1921  1933  1934  1935
  [73]  1937  1973  1974  2001  2023  2035  2053  2065  2071  2077  2091  2196
  [85]  2210  2217  2230  2233  2239  2283  2284  2300  2306  2323  2344  2399
  [97]  2436  2671  2676  2678  2703  2743  2781  3073  3149  3234  3235  3326
 [109]  3327  3328  3329  3330  3331  3332  3333  3334  3335  3336  3337  3448
 [121]  3449  3450  3451  3452  3453  3454  3455  3456  3514  3631  3634  3659
 [133]  3670  3672  3678  3700  3709  3747  3748  3749  3750  3751  3752  3753
 [145]  3754  3755  3756  3757  3758  3759  3760  3761  4121  4122  4123  4124
 [157]  4125  4126  4127  4128  4129  4130  4131  4132  4133  4134  4135  4136
 [169]  4158  4159  4160  4161  4162  4163  4164  4165  4166  4167  4168  4169
 [181]  4170  4171  4172  4173  4174  4175  4176  4177  4178  4179  4180  4181
 [193]  4182  4183  4184  4185  4186  4187  4188  4189  4190  4191  4192  4193
 [205]  4194  4195  4196  4197  4198  4199  4200  4201  4202  4203  4204  4411
 [217]  4412  4413  4415  4416  4417  4423  4442  4443  4444  4445  4446  4447
 [229]  4448  4449  4450  4451  4452  4453  4454  4455  4456  4457  4458  4459
 [241]  4460  4461  4462  4463  4464  4465  4466  4467  4468  4469  4470  4471
 [253]  4472  4473  4474  4475  4476  4477  4478  4519  4520  4523  4524  4525
 [265]  4526  4527  4528  4529  4530  4531  4532  4533  4534  4535  4536  4537
 [277]  4538  4539  4540  4541  4542  4543  4544  4545  4546  4547  4548  4549
 [289]  4550  4551  4552  4553  4554  4555  4556  4799  4813  4817  4823  4826
 [301]  4829  4831  4832  4836  4839  4840  4900  4924  4926  4946  4947  4952
 [313]  4953  4967  5032  5103  5104  5105  5117  5119  5120  5169  5178  5179
 [325]  5180  5181  5182  5183  5184  5185  5186  5187  5188  5189  5190  5307
 [337]  5308  5309  5514  5515  5516  5525  5528  5529  5530  5531  5532  5533
 [349]  5555  5559  5560  5576  5583  5584  5585  5589  5593  5594  5595  5596
 [361]  5600  5601  5602  5608  5617  5619  5631  5632  5634  5653  5656  5657
 [373]  5662  5664  5665  5679  5680  5684  5685  5686  5687  5688  5689  5691
 [385]  5703  5704  5713  5764  5770  5774  5781  5793  5795  5806  5807  5809
 [397]  5824  5825  5828  5842  5844  5854  5856  5871  5872  5886  5887  5901
 [409]  5911  5916  5917  5922  5924  5934  5935  5936  5937  5938  5939  6968
 [421]  6969  6970  6971  6972  6973  6974  6975  6976  6977  6978  6979  6980
 [433]  6981  6982  6983  6984  6985  6986  6987  6988  6989  6990  6991  6992
 [445]  6993  6994  6995  6996  6997  6998  6999  7000  7001  7002  7003  7004
 [457]  7005  7006  7007  7008  7021  7024  7026  7038  7039  7040  7041  7042
 [469]  7043  7044  7045  7046  7047  7048  7049  7050  7051  7052  7053  7054
 [481]  7055  7056  7057  7058  7059  7060  7061  7062  7063  7064  7065  7066
 [493]  7067  7068  7069  7070  7083  7101  7104  7110  7121  7122  7173  7175
 [505]  7197  7465  7466  7467  7468  7469  7470  7471  7472  7473  7474  7475
 [517]  7476  7477  7478  7479  7480  7481  7482  7483  7484  7485  7486  7487
 [529]  7488  7489  7490  7491  7492  7493  7494  7495  7496  7497  7498  7499
 [541]  7500  7501  7502  7503  7504  7505  7506  7507  7508  7509  7510  7511
 [553]  7512  7513  7514  7515  7516  7517  7518  7519  7520  7521  7522  7532
 [565]  7534  7536  7537  7538  7539  7540  7543  7544  7547  7550  7551  7552
 [577]  7555  7556  7557  7559  7564  7565  7567  7568  7569  7575  7578  7583
 [589]  7584  7585  7590  7591  7592  7593  7594  7595  7597  7600  7601  7602
 [601]  7605  7606  7607  7609  7631  7638  7639  764

In [11]:
realbestmin=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    min_value=apply(count,1,min)
    if (max(min_value,na.rm = TRUE)==0){
        realbestmin[i,]=0
        }
    else
        {
        min_value[is.na(min_value)]=100 
        realbestmin[i,]= whichmin(min_value)
    }
    }

In [12]:
table(realbestmin)

realbestmin
   0    1    2    3    4 
3275 3040 3831 5917 7937 

In [13]:
realbestmean=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    mean_value=apply(count,1,meanunique)
    if (max(mean_value,na.rm = TRUE)==0){
        realbestmean[i,]=0
        }
    else
        {
        mean_value[is.na(mean_value)]=100 
        realbestmean[i,]= whichmin(mean_value)
    }
    }

In [14]:
realbestmean

2
2
2
1
0
0
2
4
0
1
1


In [15]:
head(data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,⋯,trough,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,4,1,4,0.9492015,1.440272e-05,-3.801739,-1.3040882,-0.4829440,0.4354882,0.3478599,⋯,1,0.5299121,0.8423074,2.490487,-0.51403516,0.9484577,-0.7896943,1.7947057,0.5618988,25
2,4,1,4,0.9332456,2.024615e-05,-3.528016,-1.5740960,-0.3277471,0.3143940,0.1373249,⋯,1,0.3641703,0.8571066,2.440089,-0.29175138,0.2427210,-0.6669261,0.7095412,0.5378848,25
3,4,1,4,0.9753689,3.173153e-06,-4.202105,-1.3102058,-0.3300377,0.2486809,0.1689488,⋯,1,0.2498508,0.8961490,2.733696,-0.24714224,0.2873822,-0.7152289,0.8909252,0.6050959,25
4,4,1,4,0.9597504,7.178017e-06,-4.163362,-0.7087874,-0.2787477,0.4831910,0.1164472,⋯,1,0.4809287,0.8920654,2.726705,-0.20462399,0.2600985,-0.6048727,0.6906562,0.6227230,25
5,4,1,4,0.9770473,1.682990e-07,-5.666607,0.6009075,-0.5379779,0.5748836,0.9582223,⋯,1,0.5409435,0.5996603,2.068462,-0.14366107,2.4828596,-0.2911521,1.9555069,0.7981635,59
6,4,1,4,0.9884024,1.093815e-07,4.960507,-1.0952883,-0.1131941,0.3831215,0.6847558,⋯,1,0.5608638,0.8887061,2.856883,-0.07494233,0.7650798,-0.5923500,2.3289573,0.6350911,59


In [16]:
set.seed(100)
index = sample(2,nrow(data),replace = TRUE,prob=c(0.7,0.3))

In [17]:
train_data=data[index==1,]
test_data=data[index==2,]
train_label_min=realbestmin[index==1,]
test_label_min=realbestmin[index==2,]
train_label_mean=realbestmean[index==1,]
test_label_mean=realbestmean[index==2,]

In [18]:
head(train_data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,⋯,trough,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,4,1,4,0.9492015,1.440272e-05,-3.801739,-1.3040882,-0.4829440,0.4354882,0.3478599,⋯,1,0.5299121,0.8423074,2.490487,-0.51403516,0.9484577,-0.7896943,1.7947057,0.5618988,25
2,4,1,4,0.9332456,2.024615e-05,-3.528016,-1.5740960,-0.3277471,0.3143940,0.1373249,⋯,1,0.3641703,0.8571066,2.440089,-0.29175138,0.2427210,-0.6669261,0.7095412,0.5378848,25
3,4,1,4,0.9753689,3.173153e-06,-4.202105,-1.3102058,-0.3300377,0.2486809,0.1689488,⋯,1,0.2498508,0.8961490,2.733696,-0.24714224,0.2873822,-0.7152289,0.8909252,0.6050959,25
4,4,1,4,0.9597504,7.178017e-06,-4.163362,-0.7087874,-0.2787477,0.4831910,0.1164472,⋯,1,0.4809287,0.8920654,2.726705,-0.20462399,0.2600985,-0.6048727,0.6906562,0.6227230,25
5,4,1,4,0.9770473,1.682990e-07,-5.666607,0.6009075,-0.5379779,0.5748836,0.9582223,⋯,1,0.5409435,0.5996603,2.068462,-0.14366107,2.4828596,-0.2911521,1.9555069,0.7981635,59
6,4,1,4,0.9884024,1.093815e-07,4.960507,-1.0952883,-0.1131941,0.3831215,0.6847558,⋯,1,0.5608638,0.8887061,2.856883,-0.07494233,0.7650798,-0.5923500,2.3289573,0.6350911,59


In [19]:
end_time = Sys.time()

In [20]:
time_matrix[1,]=end_time-start_time

In [21]:
end_time-start_time

Time difference of 28.03076 secs

## Target the interval where the actual error is minimum

In [22]:
start_time = Sys.time()

In [23]:
dtrain_xg_min_reg <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label_min)) 
dtrain_xg_min_cl <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_min)) )

In [24]:
dtrain_lg_min_reg <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label_min))
dtrain_lg_min_cl <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_min)))

In [25]:
xgb_min_reg <- xgboost(data = dtrain_xg_min_reg, nround=100)

[1]	train-rmse:1.939383 
[2]	train-rmse:1.625846 
[3]	train-rmse:1.441318 
[4]	train-rmse:1.337365 
[5]	train-rmse:1.276092 
[6]	train-rmse:1.238401 
[7]	train-rmse:1.215613 
[8]	train-rmse:1.200325 
[9]	train-rmse:1.190186 
[10]	train-rmse:1.181295 
[11]	train-rmse:1.172994 
[12]	train-rmse:1.168085 
[13]	train-rmse:1.165703 
[14]	train-rmse:1.158030 
[15]	train-rmse:1.156253 
[16]	train-rmse:1.150500 
[17]	train-rmse:1.147663 
[18]	train-rmse:1.142737 
[19]	train-rmse:1.137402 
[20]	train-rmse:1.133584 
[21]	train-rmse:1.123664 
[22]	train-rmse:1.120455 
[23]	train-rmse:1.116175 
[24]	train-rmse:1.114712 
[25]	train-rmse:1.111613 
[26]	train-rmse:1.108204 
[27]	train-rmse:1.105722 
[28]	train-rmse:1.100162 
[29]	train-rmse:1.096161 
[30]	train-rmse:1.094580 
[31]	train-rmse:1.093551 
[32]	train-rmse:1.088177 
[33]	train-rmse:1.083610 
[34]	train-rmse:1.082312 
[35]	train-rmse:1.079543 
[36]	train-rmse:1.077021 
[37]	train-rmse:1.072167 
[38]	train-rmse:1.063925 
[39]	train-rmse:1.059

In [26]:
xgb_min_cl <- xgboost(data = dtrain_xg_min_cl, nround=100, objective='multi:softmax',num_class=5)

[1]	train-mlogloss:1.520953 
[2]	train-mlogloss:1.459571 
[3]	train-mlogloss:1.413169 
[4]	train-mlogloss:1.375881 
[5]	train-mlogloss:1.346454 
[6]	train-mlogloss:1.320497 
[7]	train-mlogloss:1.298422 
[8]	train-mlogloss:1.280973 
[9]	train-mlogloss:1.263005 
[10]	train-mlogloss:1.246065 
[11]	train-mlogloss:1.233099 
[12]	train-mlogloss:1.220909 
[13]	train-mlogloss:1.211117 
[14]	train-mlogloss:1.199818 
[15]	train-mlogloss:1.188800 
[16]	train-mlogloss:1.178997 
[17]	train-mlogloss:1.170803 
[18]	train-mlogloss:1.158968 
[19]	train-mlogloss:1.150891 
[20]	train-mlogloss:1.144946 
[21]	train-mlogloss:1.137436 
[22]	train-mlogloss:1.131106 
[23]	train-mlogloss:1.121876 
[24]	train-mlogloss:1.117690 
[25]	train-mlogloss:1.110465 
[26]	train-mlogloss:1.105271 
[27]	train-mlogloss:1.095573 
[28]	train-mlogloss:1.085582 
[29]	train-mlogloss:1.078275 
[30]	train-mlogloss:1.071916 
[31]	train-mlogloss:1.065427 
[32]	train-mlogloss:1.059744 
[33]	train-mlogloss:1.055397 
[34]	train-mlogloss

In [ ]:
lgb_min_reg <- lgb.train(data = dtrain_lg_min_reg, nrounds = 100)

In [ ]:
params <- list(objective = "multiclass",
               num_class = 5, 
               metric = "multi_logloss")
lgb_min_cl <- lgb.train(data = dtrain_lg_min_cl,nrounds = 100,params=params)

In [ ]:
end_time = Sys.time()
time_matrix[2,]=end_time-start_time

## Target the interval where the average error is minimum

In [ ]:

start_time = Sys.time()

In [ ]:
dtrain_xg_mean_reg <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label_mean)) 
dtrain_xg_mean_cl <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_mean)) )

In [ ]:
dtrain_lg_mean_reg <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label_mean))
dtrain_lg_mean_cl <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_mean)))

In [ ]:
xgb_mean_reg <- xgboost(data = dtrain_xg_mean_reg, nround=100)

In [ ]:
xgb_mean_cl <- xgboost(data = dtrain_xg_mean_cl, nround=100, objective='multi:softmax',num_class=5)

In [ ]:
lgb_mean_reg <- lgb.train(data = dtrain_lg_mean_reg,nrounds = 100)

In [ ]:
params <- list(objective = "multiclass",
               num_class = 5, 
               metric = "multi_logloss")
lgb_mean_cl <- lgb.train(data = dtrain_lg_mean_cl,params=params,nrounds = 100)

In [ ]:
end_time = Sys.time()
time_matrix[3,]=end_time-start_time

In [ ]:
end_time-start_time

## predict

In [ ]:
start_time = Sys.time()

In [ ]:
alldatalgb <- lgb.Dataset(data = as.matrix(data))
alldataxgb <- xgb.DMatrix(data = as.matrix(data))


In [ ]:
xgbregmin=predict(xgb_min_reg,alldataxgb)
xgbclsmin=predict(xgb_min_cl,alldataxgb)
lgbregmin=predict(lgb_min_reg,as.matrix(data))
lgbclsmin=predict(lgb_min_cl,as.matrix(data))

In [ ]:
xgbregmin

In [ ]:
xgbclsmin

In [ ]:
lgbregmin

In [ ]:
lgbclsmin

In [ ]:
datalength=dim(data)[1]
lgbclsm=matrix(lgbclsmin,m,datalength)
lgbclsminr=matrix(0,datalength,1)
for(i in 1:datalength)
    {
    lgbclsminr[i,]=which.max(lgbclsm[,i])
}
lgbclsminr=lgbclsminr-1

In [ ]:
lgbclsminr

In [ ]:
xgbregmean=predict(xgb_mean_reg,alldataxgb)
xgbclsmean=predict(xgb_mean_cl,alldataxgb)
lgbregmean=predict(lgb_mean_reg,as.matrix(data))
lgbclsmean=predict(lgb_mean_cl,as.matrix(data))

In [ ]:
lgbclsm=matrix(lgbclsmean,m,datalength)
lgbclsmeanr=matrix(0,datalength,1)
for(i in 1:datalength)
    {
    lgbclsmeanr[i,]=which.max(lgbclsm[,i])
}
lgbclsmeanr=lgbclsmeanr-1

In [ ]:
preallmin=cbind(xgbclsmin,xgbregmin)
preallmin=cbind(preallmin,lgbclsminr)
preallmin=cbind(preallmin,lgbregmin)

In [ ]:
colnames(preallmin)=c('xgbclsmin','xgbregmin','lgbclsmin','lgbregmin')
head(preallmin)

In [ ]:
preallmean=cbind(xgbclsmean,xgbregmean)
preallmean=cbind(preallmean,lgbclsmeanr)
preallmean=cbind(preallmean,lgbregmean)

In [ ]:
head(preallmean)

In [ ]:
colnames(preallmean)=c('xgbclsmean','xgbregmean','lgbclsmean','lgbregmean')
head(preallmean)

In [ ]:
end_time = Sys.time()
time_matrix[4,]=end_time-start_time

In [ ]:
result_list=list(preallmin,preallmean,time_matrix)

In [57]:
save(result_list, file = "Quarterly_ets_opt_pre_result.RData")

In [56]:
time_matrix

user_time,system_time,elapsed_time
7.950534,7.950534,7.950534
11.780916,11.780916,11.780916
11.167580,11.167580,11.167580
1.693042,1.693042,1.693042


## importance

In [57]:
# 计算特征重要性
importance_matrix <- xgb.importance(model = xgb_min_cl)

# 打印特征重要性
print(importance_matrix)


              Feature        Gain       Cover  Frequency
               <char>       <num>       <num>      <num>
 1:            length 0.116298427 0.091922008 0.06338325
 2:         curvature 0.113705183 0.094843794 0.08579414
 3:         linearity 0.096278595 0.070266015 0.07166210
 4:         seas_acf1 0.061790044 0.067959073 0.05130774
 5: seasonal_strength 0.058953526 0.058757165 0.06633622
 6:            e_acf1 0.054814456 0.068474062 0.06654714
 7:           e_acf10 0.054241369 0.062110378 0.07340224
 8:            x_acf1 0.053709281 0.057987902 0.05615904
 9:             spike 0.053621437 0.071403281 0.05668635
10:           entropy 0.052280508 0.066155323 0.06095760
11:       diff2_acf10 0.047596898 0.051292117 0.05837376
12:       diff1_acf10 0.047206600 0.062830413 0.05985024
13:             trend 0.044513062 0.049545126 0.05162413
14:        diff2_acf1 0.042341506 0.036651272 0.05136047
15:           x_acf10 0.040288056 0.046958404 0.04587640
16:        diff1_acf1 0.0401312

In [58]:
# 计算特征重要性
importance_matrix <- xgb.importance(model = xgb_min_reg)

# 打印特征重要性
print(importance_matrix)

              Feature        Gain       Cover  Frequency
               <char>       <num>       <num>      <num>
 1:            length 0.156562047 0.083958538 0.05052228
 2:         curvature 0.148126032 0.097719500 0.10168408
 3:         linearity 0.128323608 0.098287301 0.09102537
 4:         seas_acf1 0.057177968 0.057272407 0.04434023
 5: seasonal_strength 0.052347619 0.062041734 0.06288638
 6:           entropy 0.049292640 0.069965235 0.05393306
 7:           e_acf10 0.048012556 0.075978011 0.07354509
 8:             spike 0.043477076 0.046528321 0.05734385
 9:             trend 0.042955745 0.039759627 0.11255596
10:            x_acf1 0.041219530 0.059373416 0.04668514
11:       diff1_acf10 0.040811056 0.075275179 0.05158815
12:       diff2_acf10 0.040338747 0.068512197 0.04966958
13:            e_acf1 0.038573050 0.043214099 0.05883607
14:        diff2_acf1 0.036451931 0.029710888 0.04497975
15:        diff1_acf1 0.029839954 0.038093356 0.03837135
16:           x_acf10 0.0276765

In [59]:
# 计算特征重要性
importance_matrix <- xgb.importance(model = xgb_mean_cl)

# 打印特征重要性
print(importance_matrix)

              Feature        Gain       Cover  Frequency
               <char>       <num>       <num>      <num>
 1:         curvature 0.124043011 0.088726470 0.08313787
 2:         linearity 0.100371692 0.078234093 0.07740422
 3:            length 0.094341966 0.091262571 0.05999479
 4: seasonal_strength 0.063017543 0.072038908 0.07229606
 5:           entropy 0.058110041 0.085448048 0.06828251
 6:         seas_acf1 0.055213340 0.063747256 0.04842325
 7:             spike 0.054989733 0.067540747 0.05749283
 8:           e_acf10 0.054622455 0.054175344 0.07130571
 9:            e_acf1 0.053540900 0.057695962 0.06192338
10:             trend 0.052236588 0.050383851 0.05587699
11:       diff1_acf10 0.050222884 0.043590132 0.06150639
12:            x_acf1 0.049633377 0.047843811 0.05045609
13:       diff2_acf10 0.044969855 0.047748502 0.05316654
14:        diff1_acf1 0.043972629 0.048235771 0.05410477
15:        diff2_acf1 0.040143813 0.047399933 0.05061246
16:           x_acf10 0.0394884

In [60]:
# 获取特征重要性
importance_matrix <- lgb.importance(lgb_min_cl)

# 打印特征重要性
print(importance_matrix)

              Feature       Gain       Cover  Frequency
               <char>      <num>       <num>      <num>
 1:            length 0.14831070 0.181767935 0.09820000
 2:         curvature 0.12532689 0.118087604 0.08446667
 3:         linearity 0.09845920 0.069778034 0.06926667
 4:         seas_acf1 0.06364841 0.070589126 0.05826667
 5: seasonal_strength 0.05504672 0.053784997 0.06546667
 6:            e_acf1 0.05055086 0.055647431 0.06353333
 7:            x_acf1 0.05049966 0.062784288 0.05380000
 8:           e_acf10 0.05023664 0.049968993 0.06873333
 9:             spike 0.04972694 0.055177141 0.05540000
10:           entropy 0.04888326 0.045536466 0.05820000
11:       diff1_acf10 0.04544158 0.044130319 0.05593333
12:       diff2_acf10 0.04297446 0.029219265 0.05613333
13:             trend 0.03961292 0.042836804 0.04500000
14:        diff2_acf1 0.03714448 0.038061900 0.04920000
15:        diff1_acf1 0.03675990 0.034967860 0.04713333
16:           x_acf10 0.03611147 0.038265725 0.0

In [61]:
# 获取特征重要性
importance_matrix <- lgb.importance(lgb_mean_cl)

# 打印特征重要性
print(importance_matrix)

              Feature       Gain       Cover  Frequency
               <char>      <num>       <num>      <num>
 1:         curvature 0.14086553 0.110872304 0.08540000
 2:            length 0.11919675 0.168591035 0.09153333
 3:         linearity 0.10130788 0.079068342 0.07413333
 4: seasonal_strength 0.05953520 0.059359268 0.06800000
 5:         seas_acf1 0.05659529 0.069931009 0.05600000
 6:             spike 0.05428578 0.060257327 0.06033333
 7:           entropy 0.05351535 0.061761838 0.06186667
 8:            x_acf1 0.05213583 0.051058461 0.05326667
 9:            e_acf1 0.04984628 0.044560633 0.05700000
10:             trend 0.04869484 0.050770375 0.05240000
11:           e_acf10 0.04768184 0.045819089 0.06453333
12:       diff2_acf10 0.04317146 0.043341500 0.05566667
13:       diff1_acf10 0.04275537 0.026200278 0.05360000
14:        diff1_acf1 0.03694647 0.037713554 0.04700000
15:           x_acf10 0.03672010 0.041690949 0.04300000
16:        diff2_acf1 0.03545712 0.036842279 0.0

In [62]:
# 获取特征重要性
importance_matrix <- lgb.importance(lgb_mean_reg)

# 打印特征重要性
print(importance_matrix)

              Feature        Gain       Cover   Frequency
               <char>       <num>       <num>       <num>
 1:         curvature 0.223222151 0.121129995 0.086333333
 2:         linearity 0.167493789 0.118034061 0.089666667
 3:            length 0.150089767 0.198891694 0.113666667
 4:             trend 0.052597010 0.055802745 0.059666667
 5: seasonal_strength 0.050667168 0.066707060 0.071666667
 6:         seas_acf1 0.046535125 0.059116852 0.056333333
 7:           entropy 0.044534361 0.046879817 0.052333333
 8:            x_acf1 0.036214197 0.072703743 0.060000000
 9:             spike 0.034837924 0.039391173 0.050333333
10:       diff1_acf10 0.032968107 0.037316443 0.058666667
11:            e_acf1 0.030778586 0.031027051 0.051000000
12:           e_acf10 0.030345597 0.038185093 0.061666667
13:        diff2_acf1 0.022187661 0.019161579 0.042000000
14:        diff1_acf1 0.021945964 0.022483209 0.040000000
15:       diff2_acf10 0.021691057 0.018441024 0.042000000
16:           

In [63]:
importance_matrix1 <- xgb.importance(model = xgb_mean_cl)
importance_matrix2 <- xgb.importance(model = xgb_mean_reg)
importance_matrix3 <- lgb.importance(lgb_mean_cl)
importance_matrix4 <- lgb.importance(lgb_mean_reg)

In [68]:
importance_matrix1=importance_matrix1[order(importance_matrix1$Feature),]
importance_matrix2=importance_matrix2[order(importance_matrix2$Feature),]
importance_matrix3=importance_matrix3[order(importance_matrix3$Feature),]
importance_matrix4=importance_matrix4[order(importance_matrix4$Feature),]

In [71]:
importance_matrix_mean=(importance_matrix1[,2:4]+importance_matrix2[,2:4]+importance_matrix3[,2:4]+importance_matrix4[,2:4])/4

In [74]:
importance=cbind(importance_matrix1[,1],importance_matrix_mean)

In [75]:
write.csv(importance,'q_ets_imp_mean.csv')

In [67]:
importance_matrix2[order(importance_matrix2$Feature),]

Feature,Gain,Cover,Frequency
<chr>,<dbl>,<dbl>,<dbl>
curvature,0.175044521,0.100448622,0.09531879
diff1_acf1,0.030706833,0.044997740,0.04236391
diff1_acf10,0.036476061,0.047902408,0.05231942
diff2_acf1,0.033255502,0.062314556,0.04744757
diff2_acf10,0.033763447,0.050015906,0.04511756
e_acf1,0.045177900,0.073497805,0.06375768
e_acf10,0.039713684,0.068464274,0.06142766
entropy,0.055700885,0.063087407,0.05825037
length,0.103134882,0.077842398,0.04702394


In [76]:
# 使用saveRDS保存模型到.rds文件
saveRDS(xgb_mean_reg, file = "Q_etsxgbreg.rds")
saveRDS(xgb_mean_cl, file = "Q_etsxgbcls.rds")
saveRDS(lgb_mean_reg, file = "Q_etslgbreg.rds")
saveRDS(lgb_mean_cl, file = "Q_etslgbcls.rds")